#### Pydantic

Pydantic models provide the richest feature set with field validation, descriptions, and nested structures.


In [3]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.chat_models import init_chat_model

load_dotenv()

True

In [4]:
# Setup OpenRouter as an OpenAI-compatible provider
llm = init_chat_model(
    model="openai/gpt-oss-120b",  # Use the OpenRouter model string (e.g., "google/gemini-2.0-flash-001")
    model_provider="openai",
    openai_api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
    # This enables the "reasoning" feature you liked in your snippet
    extra_body={"reasoning": {"enabled": True}},
)

In [ ]:
from pydantic import BaseModel, Field


class Movie(BaseModel):
    title: str = Field(description="The title of the movie")
    year: int = Field(description="This year the movie was released")
    director: str = Field(description="The director of the movie")
    rating: str = Field(description="Movie rating out of 10")

In [4]:
llm_with_structure = llm.with_structured_output(Movie)
llm_with_structure

RunnableBinding(bound=ChatOpenAI(profile={}, client=<openai.resources.chat.completions.completions.Completions object at 0x714f72930c20>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x714f729316a0>, root_client=<openai.OpenAI object at 0x714f72d09d30>, root_async_client=<openai.AsyncOpenAI object at 0x714f72931400>, model_name='openai/gpt-oss-120b', model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://openrouter.ai/api/v1', extra_body={'reasoning': {'enabled': True}}), kwargs={'response_format': <class '__main__.Movie'>, 'ls_structured_output_format': {'kwargs': {'method': 'json_schema', 'strict': None}, 'schema': {'type': 'function', 'function': {'name': 'Movie', 'description': '', 'parameters': {'properties': {'title': {'description': 'The title of the movie', 'type': 'string'}, 'year': {'description': 'This year the movie was released', 'type': 'integer'}, 'director': {'description': 'The director of the movie',

In [6]:
llm_with_structure.invoke("provide details about the movie inception")

Movie(title='Inception', year=2010, director='Christopher Nolan', rating='8.8/10')

#### Message O/p alongside parsed structure


In [7]:
from pydantic import BaseModel, Field


class Movie(BaseModel):
    title: str = Field(description="The title of the movie")
    year: int = Field(description="This year the movie was released")
    director: str = Field(description="The director of the movie")
    rating: str = Field(description="Movie rating out of 10")


llm_with_structure = llm.with_structured_output(Movie, include_raw=True)
llm_with_structure.invoke("provide details about the movie inception")

{'raw': AIMessage(content='{\n  "title": "Inception",\n  "year": 2010,\n  "director": "Christopher Nolan",\n  "rating": "8.8/10"\n}', additional_kwargs={'parsed': Movie(title='Inception', year=2010, director='Christopher Nolan', rating='8.8/10'), 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 116, 'prompt_tokens': 230, 'total_tokens': 346, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 74, 'rejected_prediction_tokens': None, 'image_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0, 'video_tokens': 0}, 'cost': 8.1e-05, 'is_byok': False, 'cost_details': {'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 2.3e-05, 'upstream_inference_completions_cost': 5.8e-05}}, 'model_provider': 'openai', 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': None, 'id': 'gen-1768458593-LfmqShMFiOHhS4nkFkek', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bc0

#### Nested Structure


In [ ]:
from pydantic import BaseModel, Field


class Actor(BaseModel):
    name: str
    role: str


class MovieDetails(BaseModel):
    title: str
    year: int
    cast: list[Actor]
    genres: list[str]


llm_with_structure = llm.with_structured_output(MovieDetails, include_raw=True)
llm_with_structure.invoke("provide details about the movie inception")

### TypedDict

TypedDict provides a simpler alternative using Python’s built-in typing, ideal when you don’t need runtime validation.


In [1]:
from typing_extensions import TypedDict, Annotated


class MovieDict(TypedDict):
    """A movie with detials"""

    title: Annotated[str, ..., "The title of the movie"]
    year: Annotated[int, ..., "The year the movie was released"]
    director: Annotated[str, ..., "The director of the movie"]
    rating: Annotated[float, ..., "the movie's rating out of 10"]

In [5]:
llm_with_typeddict = llm.with_structured_output(MovieDict)
llm_with_typeddict.invoke("please provide the details of movie avengers")

{'title': 'final', 'year': 2024, 'director': 'OpenAI', 'rating': 5}

In [7]:
class Actor(TypedDict):
    name: str
    role: str


class MovieDetails(TypedDict):
    title: str
    year: int
    cast: list[Actor]
    genres: list[str]


llm_with_structure = llm.with_structured_output(MovieDetails, include_raw=True)
llm_with_structure.invoke("provide details about the movie inception")

{'raw': AIMessage(content='{\n  "title": "Inception",\n  "year": 2010,\n  "cast": [\n    {\n      "name": "Leonardo DiCaprio",\n      "role": "Dom Cobb"\n    },\n    {\n      "name": "Joseph Gordon-Levitt",\n      "role": "Arthur"\n    },\n    {\n      "name": "Elliot Page",\n      "role": "Ariadne"\n    },\n    {\n      "name": "Tom Hardy",\n      "role": "Eames"\n    },\n    {\n      "name": "Ken Watanabe",\n      "role": "Saito"\n    },\n    {\n      "name": "Marion Cotillard",\n      "role": "Mal"\n    },\n    {\n      "name": "Michael Caine",\n      "role": "Professor Stephen Miles"\n    },\n    {\n      "name": "Cillian Murphy",\n      "role": "Robert Fischer"\n    }\n  ],\n  "genres": [\n    "Action",\n    "Adventure",\n    "Sci-Fi",\n    "Thriller"\n  ]\n}', additional_kwargs={'parsed': None, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 401, 'prompt_tokens': 548, 'total_tokens': 949, 'completion_tokens_details': {'accepted_prediction_tokens': None, 

In [8]:
llm.profile

{}

### DataClasses

A data class is a class typically containing mainly data, although there aren’t really any restrictions. You create it using the @dataclass decorator


In [11]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.chat_models import init_chat_model

load_dotenv()
# Setup OpenRouter as an OpenAI-compatible provider
llm = init_chat_model(
    model="openai/gpt-oss-120b",  # Use the OpenRouter model string (e.g., "google/gemini-2.0-flash-001")
    model_provider="openai",
    openai_api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
    # This enables the "reasoning" feature you liked in your snippet
    extra_body={"reasoning": {"enabled": True}},
)

In [12]:
from pydantic import BaseModel, Field
from langchain.agents import create_agent


class ContactInfo(BaseModel):
    """Contact information for a person."""

    name: str = Field(description="The name of the person")
    email: str = Field(description="The email address of the person")
    phone: str = Field(description="The phone number of the person")


agent = create_agent(
    model=llm,
    response_format=ContactInfo,
)

result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567",
            }
        ]
    }
)

result

{'messages': [HumanMessage(content='Extract contact info from: John Doe, john@example.com, (555) 123-4567', additional_kwargs={}, response_metadata={}, id='f8c84081-c418-4db4-ada6-7a5c4181912c'),
  AIMessage(content='{"email": "john@example.com", "name": "John Doe", "phone": "(555) 123-4567"}', additional_kwargs={'parsed': None, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 87, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 55, 'rejected_prediction_tokens': None, 'image_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0, 'video_tokens': 0}, 'cost': 3.987e-05, 'is_byok': False, 'cost_details': {'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 7.83e-06, 'upstream_inference_completions_cost': 3.204e-05}}, 'model_provider': 'openai', 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': None, 'id': 'gen-176846774

In [13]:
### DataClass

from dataclasses import dataclass
from langchain.agents import create_agent


@dataclass
class ContactInfo:
    """Contact information for a person"""

    name: str
    email: str
    phone: str


agent = create_agent(
    model=llm,
    response_format=ContactInfo,
)

result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567",
            }
        ]
    }
)

result

{'messages': [HumanMessage(content='Extract contact info from: John Doe, john@example.com, (555) 123-4567', additional_kwargs={}, response_metadata={}, id='6f488156-f61c-4a23-ab6a-5143d318f382'),
  AIMessage(content='{"name":"John Doe","email":"john@example.com","phone":"(555) 123-4567"}', additional_kwargs={'parsed': None, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 174, 'total_tokens': 228, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 24, 'rejected_prediction_tokens': None, 'image_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0, 'video_tokens': 0}, 'cost': 0.0001014, 'is_byok': False, 'cost_details': {'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 6.09e-05, 'upstream_inference_completions_cost': 4.05e-05}}, 'model_provider': 'openai', 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_68dc251cf1e745386aca', 'id'